In [12]:
# 1. 필요한 라이브러리
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

# 2. 데이터 준비
heights = np.array([
    173, 168, 182, 165, 177, 171, 174, 169, 178, 167,
    175, 181, 170, 176, 163, 184, 172, 166, 179, 173,
    158, 165, 162, 156, 168, 159, 161, 155, 164, 166,
    160, 170, 157, 163, 162, 169, 158, 154, 165, 159,
    174, 168, 180, 172, 176, 169, 183, 167, 175, 170,
    159, 162, 166, 160, 164, 156, 158, 170, 165, 161,
    173, 177, 171, 182, 169, 175, 166, 184, 163, 178,
    162, 157, 168, 160, 166, 163, 158, 155, 169, 161,
    177, 172, 181, 168, 176, 174, 183, 170, 165, 179,
    162, 159, 167, 154, 165, 163, 160, 156, 168, 164
], dtype=np.float32)

weights = np.array([
    70.2, 65.3, 78.1, 64.5, 73.0, 67.8, 71.1, 66.5, 75.2, 63.7,
    72.0, 79.4, 66.0, 74.1, 60.5, 81.2, 69.0, 62.8, 76.5, 70.3,
    54.0, 59.5, 57.2, 52.8, 61.0, 55.4, 56.8, 51.9, 58.3, 60.2,
    55.7, 63.0, 53.4, 58.6, 57.0, 61.5, 54.2, 50.9, 59.8, 55.1,
    71.5, 66.9, 77.8, 69.0, 73.7, 66.2, 82.0, 64.0, 72.5, 67.0,
    55.5, 58.0, 61.2, 55.8, 60.7, 52.5, 54.8, 62.3, 59.0, 56.0,
    70.8, 75.5, 68.2, 79.0, 66.7, 73.5, 63.0, 81.8, 60.5, 76.0,
    57.5, 52.9, 61.8, 55.3, 61.0, 58.0, 53.5, 51.2, 60.9, 56.4,
    74.5, 69.2, 78.5, 66.0, 73.8, 71.0, 82.1, 67.0, 63.5, 77.0,
    57.0, 54.5, 61.5, 50.8, 59.9, 58.2, 55.0, 52.0, 62.5, 60.0
], dtype=np.float32)

labels = np.array([
    1,1,1,1,1,1,1,1,1,1,
    1,1,1,1,1,1,1,1,1,1,
    0,0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,0,
    1,1,1,1,1,1,1,1,1,1,
    0,0,0,0,0,0,0,0,0,0,
    1,1,1,1,1,1,1,1,1,1,
    0,0,0,0,0,0,0,0,0,0,
    1,1,1,1,1,1,1,1,1,1,
    0,0,0,0,0,0,0,0,0,0
], dtype=np.float32)

# 2D input: height + weight
x = np.column_stack((heights, weights))
y = labels

print(x[20])
print(y[20])

[158.  54.]
0.0


In [13]:
# ---------------------------------------------
# 2. 학습용(train) / 테스트용(test) 데이터 나누기
# ---------------------------------------------

# train_test_split 함수는 데이터를 무작위로 섞은 뒤
# 80% → 학습용, 20% → 테스트용으로 분리함
# random_state는 '무작위 시드'로, 항상 같은 결과를 얻기 위해 고정함
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 모델 Tensor 타입으로 변환
# float32로 자동 변환됨
x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)
x_test = torch.tensor(x_test)
y_test = torch.tensor(y_test)
"""
차원	형태	예시
0D	[5]	숫자 하나
1D	[1,2,3]	벡터
2D	[[1,2],[3,4]]	행렬
3D	[행렬 여러개]	이미지 1장
4D	[이미지 여러장]	이미지 batch
5D	[영상]	video (time 포함)
6D+	딥러닝 내부 연산	의미 가능
"""

# 배치
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


In [14]:
# 3. 모델 정의 (간단한 신경망)
class GenderClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.layer(x)

model = GenderClassifier()


In [15]:
# 4. Loss 함수
criterion = nn.BCELoss()

docs = """
[틀린 비율 측정]
- MSE: 회귀
ㄴ MSE = 평균((예측 - 정답)^2)

- Cross Entropy: 분류, LLM
ㄴ CE = - Σ y * log(p)
ㄴ p = 정답 확률

- L1 Loss: 노이즈(이상치)가 심할때 주로 사용
ㄴ L1 = 평균(|예측 - 정답|)

- KL Divergence: 분포 차이 측정
ㄴ KL = Σ P(x) * log(P(x) / Q(x))
ㄴ P = 정답 분포, Q = 모델의 예측 분포

- BCE Loss: 이진 분류용
ㄴ BCE = - [ y*log(p) + (1−y)*log(1−p) ]
ㄴ p = 모델의 예측 확률, y = 정답

"""


In [16]:
# 5. Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
docs = """
[최적화 방법론]

- Adam:
ㄴ 자주 같은 방향으로 gradient가 나오면 그 방향으로 더 강하게 움직인다(관성).
ㄴ gradient가 너무 큰 파라미터는 줄이고, 작은 파라미터는 더 크게 업데이트한다.

- AdamW
ㄴ 너무 큰 가중치를 자동으로 줄여서 과적합(overfitting)을 완화한다.
"""

# Learning Rate Scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
docs = """
어떤 값을 시간에 따라 조절하는 장치
없을경우 최적의 지점에서 발산

1) StepLR
 - 일정 epoch마다 학습률을 gamma 배로 감소시키는 방식.
 - lr = lr0 * (gamma ^ floor(epoch / step_size))

2) MultiStepLR
 - 여러 지정된 epoch 구간마다 학습률 감소.
 - 예: milestones=[30, 60, 90]

3) ExponentialLR
 - 매 epoch마다 학습률을 gamma 비율만큼 감소.
 - lr = lr0 * (gamma ^ epoch)

4) CosineAnnealingLR
 - cosine 곡선을 따라 학습률을 부드럽게 감소.
 - 중후반에 자연스럽게 0으로 수렴.
 - 최신 모델(Transformer, Diffusion)에서 자주 사용.


"""

In [22]:
# 6. Training
epochs = 50
for epoch in range(epochs):
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()

        # 예측값 생성 ( 순전파 )
        preds = model(batch_x).squeeze()
        # LOSS 계산
        loss = criterion(preds, batch_y)

        # 모델의 정확도가 높은 방향을 지정 (역전파)
        loss.backward()

        # Optimizer가 파라미터 업데이트
        optimizer.step()

    # 스케줄러로 학습 속도 조절
    scheduler.step()

    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}", )

Epoch 10, Loss: 0.1427
Epoch 20, Loss: 0.1091
Epoch 30, Loss: 0.0171
Epoch 40, Loss: 0.1059
Epoch 50, Loss: 0.1588


In [23]:
# 7. 평가
with torch.no_grad():
    preds = model(x_test).squeeze()
    preds = (preds > 0.5).float()
    accuracy = (preds == y_test).float().mean()

print(f"\n테스트 정확도: {accuracy.item()*100:.2f}%")


테스트 정확도: 100.00%


In [31]:
# 8. 새로운 데이터 예측
new_height = torch.tensor([[168.0, 75]])
new_pred = model(new_height).item()

print("남자" if new_pred > 0.5 else "여자")


남자
